---
title: Nuestro primer modelo en Pyomo
subtitle: Un problema de planificación sencillo
---

[![Launch on Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/boiro9/jupyterbook2_example/HEAD?labpath=021_Pyomo_ejemplo_sencillo.ipynb)[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/boiro9/jupyterbook2_example/blob/HEAD/021_Pyomo_ejemplo_sencillo.ipynb)

## Problema de planificación

Una empresa produce dos versiones de un producto. Cada versión se fabrica a partir de la misma materia prima, que cuesta 10 por gramo, y cada versión requiere dos tipos diferentes de mano de obra especializada para finalizarla. 

**U** es la versión más cara del producto. Se vende a 270 por unidad y requiere 10 gramos de materia prima, 1 hora de mano de obra tipo **A** y 2 horas de mano de obra tipo **B**. Debido a su mayor precio, la demanda de U está limitada a **40 unidades por semana**.

**V** es la versión más económica, con demanda ilimitada. Se vende a 210 por unidad y requiere 9 gramos de materia prima, 1 hora de mano de obra tipo **A** y 1 hora de mano de obra tipo **B**.

Esta información se resume en la siguiente tabla:

| Versión | Materia prima <br> requerida | Mano de obra A <br> requerida | Mano de obra B <br> requerida | Demanda <br> de mercado | Precio |
| :-: | :-: | :-: | :-: | :-: | :-: | 
| U | 10 g | 1 h | 2 h | $\leq$ 40 unidades | 270 |
| V |  9 g | 1 h | 1 h | ilimitada | 210 |


La producción semanal de la empresa está limitada por la disponibilidad de mano de obra y el inventario de materia prima. La materia prima tiene una **vida útil de una semana** y debe ser pedida por adelantado. Cualquier materia prima sobrante al final de la semana se descarta.

La siguiente tabla detalla el coste y la disponibilidad de la materia prima y la mano de obra:

| Recurso | Cantidad <br> disponible | Coste |
| :-: | :-: | :-: |
| Materia prima | ilimitada | 10 / g |
| Mano de obra A | 80 horas/semana | 50 / h |
| Mano de obra B | 100 horas/semana | 40 / h |

La empresa desea **maximizar sus beneficios brutos**.
1. ¿Cuánta materia prima debería pedirse por adelantado cada semana?  
2. ¿Cuántas unidades de **U** y **V** debería producir la empresa cada semana?


Este problema puede formularse como un problema de programación matemática. Para ello debemos de definir:
- Las variables de decisión del problema.
- La función objetivo.
- El conjunto de restricciones.

En este caso, podemos definir las siguientes variables de decisión:

<div align="center">

| Variable de decisión | Descripción | límite inferior | límite superior |
| :-: | :--- | :-: | :-: |
| $x_M$ | cantidad de materia prima utilizada | 0 | - |
| $x_A$ | cantidad de mano de obra A utilizada | 0 | 80 |
| $x_B$ | cantidad de mano de obra B utilizada | 0 | 100 |
| $y_U$ | número de unidades de $U$ a producir | 0 | 40 |
| $y_V$ | número de unidades de $V$ a producir | 0 | - |

</div>

En cuanto a la **función objetivo**, en este caso queremos maximizar los beneficios obtenidos, es decir, la diferencia entre los beneficios y los costes. Si definimos las siguientes **expresiones** (lineales):
$$
\begin{aligned}
    \text{beneficios} & = 270 y_U + 210 y_V \\
    \text{costes}     & = 10 x_M + 50 x_A + 40 x_B  \\
\end{aligned}
$$
la función objetivo del problema sería:
$$
\text{Maximizar} \,\, \text{beneficios}-\text{costes}
$$
Por último, definamos las restricciones asociadas al problema. En este caso, para cada recurso tenemos una restricción que limita su capacidad de producción:
$$
\begin{aligned}
    10 y_U + 9 y_V  & \leq x_M & & \text{(materia prima)}\\
    1 y_U + 1 y_V & \leq x_A & &\text{(mano de obra A)} \\
    2 y_U + 1 y_V & \leq x_B & & \text{(mano de obra B)}\\
\end{aligned}
$$
Resumiendo, la formulación matemática del problema quedaría como sigue:

$$
\begin{align}
\max \quad & 270 y_U + 210 y_V - 10 x_M - 50 x_A - 40 x_B \\
\text{s.a.} \quad & 10 y_U + 9 y_V  \leq x_M \nonumber \\
 & 1 y_U + 1 y_V \leq x_A \nonumber \\
 & 2 y_U + 1 y_V \leq x_B \nonumber \\
 & 0 \leq x_M \nonumber \\
 & 0 \leq x_A \leq 80 \nonumber \\
 & 0 \leq x_B \leq 100 \nonumber \\
 & 0 \leq y_U \leq 40 \nonumber \\
 & 0 \leq y_V.\nonumber 
\end{align}
$$



## Resolución del problema con Pyomo

En esta sección resolveremos el sencillo problema de producción introducido previamente con Python. Los principales elementos de Pyomo que necesitaremos son los siguientes:

* [Variables](https://pyomo.readthedocs.io/en/6.8.0/pyomo_modeling_components/Variables.html)
* [Expressions](https://pyomo.readthedocs.io/en/6.8.0/pyomo_modeling_components/Expressions.html)
* [Objectives](https://pyomo.readthedocs.io/en/6.8.0/pyomo_modeling_components/Objectives.html)
* [Constraints](https://pyomo.readthedocs.io/en/6.8.0/pyomo_modeling_components/Constraints.html)
* [SolverFactory](https://pyomo.readthedocs.io/en/6.8.0/solving_pyomo_models.html)

### 1️⃣ Importamos la librería y creamos el objeto modelo

En primer lugar tenemos tenemos que importar la librería de Pyomo en Python, para tener acceso a todas sus componentes:

In [1]:
import pyomo.environ as pyo

A continuación creamos un modelo de Pyomo especificando un nombre para el mismo. En este caso crearemos un ``ConcreteModel``, que se usa para crear el objeto modelo cuando los datos del problema ya están disponibes en el momento de contrucción del problema.
```{note} Nota
``ConcreteModel`` es la opción recomendada por los desarrolladores de Pyomo. La otra opción sería crear un ``AbstractModel``, lo cual explicaremos más adelante.
```


In [2]:
# Creamos el modelo especificando un nombre (opcional)
model = pyo.ConcreteModel("Problema de produccion 1")

El método ``.display()`` permite ver el contenido actual que está dentro del modelo de Pyomo. Este método es muy útil cuando se formulan nuevos modelos, para comprobar que el modelo se crea correctamente.

In [3]:
# Vemos en contenido del modelo
model.display()

Model Problema de produccion 1

  Variables:
    None

  Objectives:
    None

  Constraints:
    None


### 2️⃣ Variables de decisión

Ahora vamos a definir las **variables de decisión** del modelo, las cuales se crean con ``pyo.Var()``. A la hora de definir las variables podemos especificar los siguientes argumentos:
- ``domain``: especifica el dominio de la variable. Por defecto, el dominino es $\mathbb{R}$. Otros dominios que se pueden especificar: `pyo.NonNegativeReals`, `pyo.NonNegativeIntegers`, and `pyo.Binary`.
- `bounds`: argumento opcional para especificar las cotas inferior y superior de la variable. `None` puede utilizarse como valor cuando alguna de las cotas es desconocida. Por ejemplo, si se especifica la cota como `(0, None)` es equivalente a poner como dominio `pyo.NonNegativeReals`.


In [4]:
# Creamos las variables de decisión
model.x_M = pyo.Var(bounds=(0, None))
model.x_A = pyo.Var(bounds=(0, 80))
model.x_B = pyo.Var(bounds=(0, 100))

model.y_U = pyo.Var(bounds=(0, 40))
model.y_V = pyo.Var(bounds=(0, None))

# Actualizamos el modelo
model.display()

Model Problema de produccion 1

  Variables:
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    80 : False :  True :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   100 : False :  True :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    40 : False :  True :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

  Objectives:
    None

  Constraints:
    None


### 3️⃣ Función objetivo

Definimos ahora la **función objetivo**, para lo cual emplearemos en concepto de expresión de Pyomo, que no son más que fórmulas que utilizan variables de decisión. Empezamos creando expresiones para calcular los beneficios y los costes.

In [5]:
# Creamos las expresiones
model.ingresos = 270 * model.y_U + 210 * model.y_V
model.costes = 10 * model.x_M + 50 * model.x_A + 40 * model.x_B

# expressions can be printed
print(model.ingresos)
print(model.costes)

270*y_U + 210*y_V
10*x_M + 50*x_A + 40*x_B


Ahora ya podemos definir la función objetivo empleando el método `pyo.Objective`, donde la expresión a optimizar se asigna con el argumento `expr` y el sentido de la optimización `sense`.

```python
model.beneficios = pyo.Objective(expr=model.ingresos - model.costes, sense=pyo.maximize)
```

Lo anterior puede ser escrito de manera equivalente empleando decoradores de Python:

In [6]:
@model.Objective(sense=pyo.maximize)
def beneficio(m):
    return m.ingresos - m.costes

```{admonition} Decoradores en Pyomo
:class: info

Los **decoradores en Pyomo** permiten mejorar la legibilidad y simplificar la creación de modelos complejos. Sus principales características son:
- Permiten insertar funciones en un modelo de Pyomo para que actúen como: **expresiones**, **objetivos** o **restricciones**.  
- Mejoran la **legibilidad** y la **mantenibilidad** de modelos complejos.
- Simplifican la **sintaxis** para crear otros objetos de Pyomo
```

### 4️⃣ Restricciones

Ahora vamos a definir las **restricciones**. En Pyomo una restricción consiste en dos expresiones separadas por una condición lógica que las relaciona. La condición lógica puede ser de igualdad (`==`), de menor que (`<=`) o de mayor que (`>=`). En Pyomo se definen las restricciones empleando el método `pyo.Constraint()` pasando la restricción con el argumento `expr`. Para nuestro problema tenemos:

```python
model.materia_prima = pyo.Constraint(expr = 10 * model.y_U + 9 * model.y_V <= model.x_M)
model.mano_A = pyo.Constraint(expr = 1 * model.y_U + 1 * model.y_V <= model.x_A)
model.mano_B = pyo.Constraint(expr = 2 * model.y_U + 1 * model.y_V <= model.x_B)
```

Alternativamente, también se pueden definir las restricciones empleando el decorador `@model.Constraint()`.

In [7]:
@model.Constraint()
def materia_prima(m):
    return 10 * m.y_U + 9 * m.y_V <= m.x_M


@model.Constraint()
def mano_A(m):
    return 1 * m.y_U + 1 * m.y_V <= m.x_A


@model.Constraint()
def mano_B(m):
    return 2 * m.y_U + 1 * m.y_V <= m.x_B

# Imprimimos el modelo
model.pprint()

5 Var Declarations
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    80 : False :  True :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   100 : False :  True :  Reals
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    40 : False :  True :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    beneficio : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)

3 Constraint 

### 5️⃣ Resolvemos el modelo con un solver

Una vez especificado completamente el modelo, es siguiente paso es calcular una solución. Para ello es necesario llamar un solver que lleve a cabo su resolución. En Pyomo, primero se debe crear el objeto solver con `SolverFactory`, donde se configuran todos los aspectos del mismo, y a continuación aplicárselo al modelo para que lo resuelva. El argumento opcional `tee=True` hace que el solver imprima por pantalla sus cálculo, lo cual es útil en fases de desarrollo. A continuación resolvemos el problema empleando el solver HIGHS.

In [8]:
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(model, tee=True)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 3 rows; 5 cols; 9 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+01, 3e+02]
  Bound  [4e+01, 1e+02]
  RHS    [0e+00, 0e+00]
Presolving model
2 rows, 4 cols, 6 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
2 rows, 4 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-1); columns 4(-1); elements 6(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999953995e+02 Ph1: 2(4); Du: 3(210) 0s
          3    -2.6000000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  2.6000000000e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.

````{warning} 
El solver que querramos usar tiene que estar previamente instalado. Por ejempo, en este caso que estamos usando el solver **HIGHS**, podemos instalarlo ejecutando:  

```cmd
pip install highspy 
```

Una opción útil para acceder a los solvers es emplear los ejecutables que ya tiene preparados **AMPL** para poder utilizarlos con Python. En el apartado `Using from Pyomo` del siguiente [link](https://dev.ampl.com/ampl/python/modules.html), aparece explicado como instalarlos y usarlos.

Para instalarlos:

```cmd
pip install amplpy pyomo -q
python -m amplpy.modules install coin highs scip gcg -q  # Install HiGHS, CBC, Couenne, Bonmin, Ipopt, SCIP, and GCG
```

Ejemplo de como usarlo en Pyomo:

```python
from amplpy import modules
import pyomo.environ as pyo
solver_name = "highs"  # "highs", "cbc",  "couenne", "bonmin", "ipopt", "scip", or "gcg".
solver = pyo.SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")

model = pyo.ConcreteModel()
model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr = 2*model.x[1] + 3*model.x[2])
model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)
solver.solve(model, tee=True)
```

````

### 6️⃣ Reportamos la solución

Por último, tendríamos que explorar la **solución obtenida**. Para ellos podemos hacer uso del método `pprint()` o `display`, que se puede aplicar tanto al modelo completo como a alguna componente individual del mismo. Por ejemplo, en el campo `value` de las variables podemos ver el valor "óptimo" que toman las variables de acuerdo a la solución que ha encontrado el solver.

In [9]:
# Mostramos el modelo completo:
model.pprint()

5 Var Declarations
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  80.0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 100.0 :   100 : False : False :  Reals
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 740.0 :  None : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False :  Reals

1 Objective Declarations
    beneficio : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)

3 Constraint 

In [10]:
# Mostramos un componente específico del modelo:
model.x_A.pprint()

x_A : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  80.0 :    80 : False : False :  Reals


In [11]:
# También se puede emplear el método display
model.beneficio.display()

beneficio : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 2600.0


Una vez se ha obtenido la solución de un modelo de Pyomo, se puede acceder a los valores de la función objetivo, las expresiones y las variables de decisión con `pyo.value()`.

In [12]:
print(f" Profit = {pyo.value(model.beneficio): 9.2f}")
print(f"Revenue = {pyo.value(model.ingresos): 9.2f}")
print(f"   Cost = {pyo.value(model.costes): 9.2f}")

 Profit =   2600.00
Revenue =  18000.00
   Cost =  15400.00


Al valor de las variables también se puede acceder de manera más directa haciendo:

In [13]:
print("x_A =", model.x_A())
print("x_B =", model.x_B())
print("x_M =", model.x_M())

x_A = 80.0
x_B = 100.0
x_M = 740.0


# 🧠 Ejercicios propuestos

:::{exercise}
:label: ex:pyomo1

Resuelve el siguiente problema de optimización empleando Pyomo:

\begin{align*}
\textrm{Maximizar}\ \ \     &  3x+y\\
\textrm{sujeto a}\ \ \      & x+y\leq 2\\
                            & 0\leq x \leq 1\\
                 \ \ \       & 0\leq y \leq 2\\                     
\end{align*}

````{solution} ex:pyomo1
:label: sol:pyomo1
:class: dropdown

```{code-block} python
import pyomo.environ as pyo

# Crear el modelo
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(bounds=(0, 1))   # 0 <= x <= 1
model.y = pyo.Var(bounds=(0, 2))   # 0 <= y <= 2

# Función objetivo: Maximizar 3x + y
model.obj = pyo.Objective(expr=3*model.x + model.y, sense=pyo.maximize)

# Restricciones
model.restriccion1 = pyo.Constraint(expr=model.x + model.y <= 2)

# Resolver
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(model, tee=True)

# Mostrar resultados
print(f"x = {pyo.value(model.x)}")
print(f"y = {pyo.value(model.y)}")
print(f"Objetivo máximo = {pyo.value(model.obj)}")
    
```

````

:::


In [14]:
# Tu código aquí

:::{exercise}
:label: ex:pyomo2

Un deportista se encuentra preparando una maratón, para lo cual dispone de un preparador físico que le ha planificado 3 posibles ejercicios físicos de trabajo.  
 1. El deportista debe decidir qué ejercicios llevar a cabo, maximizando el rendimiento físico que le aportan.
    - La actividad 1 y 2 le aportan un 1% de mejora.
    - La actividad 3 un 2%.
 2. El deportista dispone de 4 horas para llevar a cabo el entrenamiento:
    - La actividad 1 requiere 1 hora.
    - La actividad 2 requiere 2 horas.
    - La actividad 3 requiere 3 horas.
 3. El preparador físico le ha obligado a que realice la actividad 1 o 2 (o ambas).

\begin{align*}
\textrm{Maximizar}\ \ \     &  x+y+2z\\
\textrm{sujeto a}\ \ \      & x+2y+3z\leq 4\\
                            & x+y\geq 1\\
                 \ \ \      & x,y,z\in \{0,1\}\\                     
\end{align*}

Resuelve el problema de optimización empleando Pyomo.

````{solution} ex:pyomo2
:label: sol:pyomo2
:class: dropdown

```{code-block} python
import pyomo.environ as pyo

# Crear el modelo
model = pyo.ConcreteModel()

# Variables binarias
model.x = pyo.Var(domain=pyo.Binary)
model.y = pyo.Var(domain=pyo.Binary)
model.z = pyo.Var(domain=pyo.Binary)

# Función objetivo: Maximizar x + y + 2z
model.obj = pyo.Objective(expr=model.x + model.y + 2*model.z, sense=pyo.maximize)

# Restricciones
model.restriccion1 = pyo.Constraint(expr=model.x + 2*model.y + 3*model.z <= 4)
model.restriccion2 = pyo.Constraint(expr=model.x + model.y >= 1)

# Resolver
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
results = SOLVER.solve(model, tee=True)

# Mostrar resultados
print(f"x = {pyo.value(model.x)}")
print(f"y = {pyo.value(model.y)}")
print(f"z = {pyo.value(model.z)}")
print(f"Objetivo máximo = {pyo.value(model.obj)}")
```

````

:::

In [15]:
# Tu código aquí